# Introduction to CUDA Quantum

    Outline 

    1. What is CUDA Quantum? 
    2. CUDA Quantum Kernels
    3. CUDA Quantum Primitives
        3.1 cudaq.sample() 
        3.2 cudaq.spin_op()
        3.3 cudaq.observe()
    4. Parameterized circuits 
    5. Noise-modeling
    6. Miscellaneous
        6.1. Target physical qpus
        6.2. Adjoint of a kernel
        6.3. Conditional measurement
        6.4. Variational quantum algorithm two ways
    7. Single-gpu acceleration

### 1. CUDA Quantum

    - NVIDIA’s open-source platform for hybrid quantum-classical computing 

    - Built for high-performance, scalability, and ease-of-use

    - QPU Agnostic : Partnering broadly including superconducting, trapped ion, neutral atom, photonic, and NV center QPUs​

    - Interoperable with the modern scientific computing ecosystem​

    - Retargetable : seamless transition from simulation to physical QPU

    - As all valuable quantum applications of the future will be hybrid, CUDA Quantum enables users to develop performant hybrid applications that can easily scale to supercomputing scale.



#### 2. CUDA Quantum Kernel

      A common pattern in the design and implementation of accelerated-node programming models is the separation of accelerator-device code from existing CPU host code via function-level boundaries.

      Quantum kernels are C++/Python functions that, when called, are executed on a quantum process unit (or a simulator). 

In [2]:
# Import the CUDA Quantum module
import cudaq

In [3]:
# We begin by defining the `Kernel` that we will construct our
# program with.
kernel = cudaq.make_kernel()

In [4]:
# Next, we can allocate qubits to the kernel via `qalloc(qubit_count)`.
# An empty call to `qalloc` will return a single qubit.
qubit = kernel.qalloc()

In [5]:
# Now we can begin adding instructions to apply to this qubit!
# Here we'll just add a selection of single qubit gates
kernel.h(qubit)
kernel.x(qubit)
kernel.y(qubit)
kernel.z(qubit)
kernel.t(qubit)
kernel.s(qubit)

In [6]:
# Next, we add a measurement to the kernel so that we can sample
# the measurement results on our simulator!
kernel.mz(qubit)

In [7]:
# Other methods and attributes available to the kernel object
dir(kernel)
#help(kernel.tdg)

['_PyKernel__addAllCalledFunctionsRecursively',
 '_PyKernel__applyControlOrAdjoint',
 '_PyKernel__cloneOrGetFunction',
 '_PyKernel__createQuakeValue',
 '_PyKernel__getMLIRValueFromPythonArg',
 '_PyKernel__isPauliWordType',
 '_PyKernel__processArgType',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'adjoint',
 'apply_call',
 'argument_count',
 'arguments',
 'c_if',
 'ch',
 'control',
 'cr1',
 'createInvariantForLoop',
 'crx',
 'cry',
 'crz',
 'cs',
 'cswap',
 'ct',
 'ctx',
 'cu3',
 'cx',
 'cy',
 'cz',
 'exp_pauli',
 'fermionic_swap',
 'for_loop',
 'from_state',
 'funcName',
 'funcNameEntryPoint',
 'funcOp',
 'getConstantFloat',
 'getConstantInt',
 'getInt

###     3. Algorithmic primitives

      Algorithmic primitives are common programming patterns that have 
      been implemented in the CUDA Quantum library.

    3.1 cudaq.sample()
    3.2 cudaq.observe()
    3.3 cudaq.spin_op()

#### 3.1. cudaq.sample()

      The sample() function performs multiple measurements of the 
      circuit(1000 shots by default) and returns a dictionary of the
      measurement outcomes along with their respective counts. 

In [8]:
# Finally, we can execute this kernel on the state vector simulator
# by calling `cudaq.sample`. This will execute the provided kernel
# `shots_count` number of times and return the sampled distribution
# as a `cudaq.SampleResult` dictionary.
sample_result = cudaq.sample(kernel)

# Now let's take a look at the `SampleResult` we've gotten back!
print(sample_result)  # or result.dump()   

{ 0:520 1:480 }



    Putting it all together!

In [9]:
import cudaq

kernel = cudaq.make_kernel()
qubit = kernel.qalloc(2)
                                  
kernel.h(qubit[0])
kernel.cx(qubit[0], qubit[1])

kernel.mz(qubit)

sample_result = cudaq.sample(kernel, shots_count=2000) 

print(sample_result)  # or sample_result.dump() 

# Extracting data from sample
print(f"most probable = {sample_result.most_probable()}")
print(f"count for 11 = {sample_result.count('11')}")
print(f"probability for 11 = {sample_result.probability('11')}") # i.e. probability from sample
print(f"Marginal counts for qubit 0 = {sample_result.get_marginal_counts([0])}")

{ 00:1025 11:975 }

most probable = 00
count for 11 = 975
probability for 11 = 0.4875
Marginal counts for qubit 0 = { 1:975 0:1025 }



In [10]:
# clear results, result should now be empty
sample_result.clear()
print(sample_result)

{ }



####  3.2. cudaq.spin_op()

     
    The spin_op represents sum of Pauli tensor products. 
    
    - Typical algebraic operations can be used to compose larger, more complex Pauli tensor products and their sums. 

Let's take the Hamitonian H such that, H  = $Z_0 \otimes I_1 + I_0 \otimes X_1 + Y_0 \otimes I_1 + Y_0 \otimes Y_1$.

In [11]:
# Importing the spin_op
from cudaq import spin

# the obseravle 
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0) + spin.y(0)*spin.y(1)

# add some more terms
for i in range(2):
  hamiltonian += -2.0*spin.z(i)*spin.z(i+1)

print(hamiltonian)
print(hamiltonian.to_matrix())
print(hamiltonian.to_sparse_matrix())

[-2+0j] IZZ
[1+0j] ZII
[1+0j] YII
[1+0j] IXI
[1+0j] YYI
[-2+0j] ZZI

(-3,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0) (-1,0)  (0,0)
 (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0) (-1,0)
 (1,0)  (0,0)  (5,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0)
 (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (0,1)
(0,-1)  (0,0)  (1,0)  (0,0) (-1,0)  (0,0)  (1,0)  (0,0)
 (0,0) (0,-1)  (0,0)  (1,0)  (0,0)  (3,0)  (0,0)  (1,0)
(-1,0)  (0,0) (0,-1)  (0,0)  (1,0)  (0,0) (-1,0)  (0,0)
 (0,0) (-1,0)  (0,0) (0,-1)  (0,0)  (1,0)  (0,0) (-5,0)

([(-3+0j), (1+0j), 1j, (-1+0j), (1+0j), (1+0j), 1j, (-1+0j), (1+0j), (5+0j), (1+0j), 1j, (1+0j), (1+0j), (1+0j), 1j, -1j, (1+0j), (-1+0j), (1+0j), -1j, (1+0j), (3+0j), (1+0j), (-1+0j), -1j, (1+0j), (-1+0j), (-1+0j), -1j, (1+0j), (-5+0j)], [0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7], [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7])


In [12]:
dir(hamiltonian)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'distribute_terms',
 'dump',
 'for_each_pauli',
 'for_each_term',
 'from_word',
 'get_coefficient',
 'get_qubit_count',
 'get_raw_data',
 'get_term_count',
 'is_identity',
 'random',
 'serialize',
 'to_matrix',
 'to_sparse_matrix',
 'to_string']

#### 3.3. cudaq.observe()

Compute the expected value of the observable, i.e., $\bra{\psi}H\ket{\psi}$, where $H$ is of type spin_op.

In [13]:
# the observable 
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0) + spin.y(0)*spin.y(1)

# First we need to construct a cuda quantum kernel
kernel = cudaq.make_kernel()
qreg = kernel.qalloc(2)
kernel.x(qreg[0])

# The cudaq.observe() takes the quantum circuit and the observable as input params
observe_result = cudaq.observe(kernel, hamiltonian, shots_count=1000)
print(observe_result)
observe_result.expectation()

-1.034

In [14]:
# For a complete list of attributes
dir(observe_result)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'counts',
 'dump',
 'expectation',
 'expectation_z',
 'get_spin']

### 4. Parameterized circuits
    A quantum circuit parameterized by one or many parameters.

In [15]:
# Example of a circuit with a single parameter of type float.
import cudaq
from cudaq import spin

# the observable 
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) \
            - 2.1433 * spin.y(0) * spin.y(1) + 0.21829 * spin.z(0) \
            - 6.125 * spin.z(1)

# parameterized cudaq kernel, the parameter is of type float
kernel, theta = cudaq.make_kernel(float)
q = kernel.qalloc(2)
kernel.x(q[0])
kernel.ry(theta, q[1])
kernel.cx(q[1], q[0])

# observe() takes the kernel, the observable and the kernel parameters
observe_result = cudaq.observe(kernel, hamiltonian, .59)
observe_result.expectation()

-1.7487948611472124

In [16]:
# Example of a circuit with many parameters passed as a list.
import cudaq
from cudaq import spin

# the observable 
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) \
            - 2.1433 * spin.y(0) * spin.y(1) + 0.21829 * spin.z(0) \
            - 6.125 * spin.z(1)

# parameterized cudaq kernel, the parameter is of type list
kernel, theta = cudaq.make_kernel(list)
q = kernel.qalloc(2)
kernel.x(q[0])
kernel.ry(theta[0], q[1])
kernel.ry(theta[1], q[1])
kernel.cx(q[1], q[0])

# observe() takes the kernel, the observable and the kernel parameters
observe_result = cudaq.observe(kernel, hamiltonian, [.59, .75])
observe_result.expectation()

0.28301576532552275

### 5. Noise modeling

    A single-qubit bit-flip error can be expressed as:

\begin{equation*}
    \rho = (1-p) \rho + p X\rho X 
\end{equation*}
    with p in [0,1].

In [17]:
import cudaq

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# We will begin by defining an empty noise model that we will add
# these decoherence channels to.
noise = cudaq.NoiseModel()

# Bit flip channel with `1.0` probability of the qubit flipping 180 degrees.
bit_flip = cudaq.BitFlipChannel(.1)
# We will apply this channel to any X gate on the qubit, giving each X-gate
# a probability of `1.0` of undergoing an extra X-gate.
noise.add_channel('x', [0], bit_flip)

# construct a circuit
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Apply an X-gate to the qubit.
# It will remain in the |1> state with a probability of `1 - p = 0.9`.
kernel.x(qubit)
kernel.mz(qubit)

# noisy simulation

noisy_result = cudaq.sample(kernel, noise_model=noise)
noisy_result.dump()

# noiseless simulation

noiseless_result = cudaq.sample(kernel)
noiseless_result.dump()

{ 0:89 1:911 }
{ 1:1000 }


### 6. Miscellaneous

    6.1. Target Physical QPUs

In [18]:
#cudaq.set_target('quantinuum')
#cudaq.set_target('quantinuum', machine='H1-2')
#cudaq.sample(kernel, shots_count=10000)

In [19]:
# Before moving on, let's reset to the default cpu backend.
cudaq.set_target("qpp-cpu") 

    6.2. Adjoint of a kernel

In [20]:
import cudaq 

# Create a kernel and do some operations
other_kernel = cudaq.make_kernel()
other_qubit = other_kernel.qalloc()
other_kernel.x(other_qubit)

# Create a kernel, which'll be the adjoint of other_kernel 
kernel = cudaq.make_kernel()
kernel.adjoint(other_kernel)

    6.3. Conditional Measurement

In [21]:
# The conditional measurement functionality of `cudaq.kernel`
import cudaq 

kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

def then_function():
    kernel.x(qubit)

kernel.x(qubit)

# Measure the qubit.
measurement_ = kernel.mz(qubit)
# Apply `then_function` to the `kernel` if
# the qubit was measured in the 1-state.
kernel.c_if(measurement_, then_function)

# Measure the qubit again.
result = cudaq.sample(kernel, shots_count=30)
print(result)

{ 0:30 }



    6.4. Variational Algorithms

    Variational algorithms in CUDA Quantum typically leverage the `cudaq.observe(...)` function in tandem with the `cudaq.optimizer`.

    One can choose an optimization strategy provided as specific sub-types of the `cudaq.optimizer`.

    The `cudaq.optimizer` can be replaced with any optimizer of your choice including the ones from scipy!

In [22]:
# Import the necessary modules
import cudaq
from cudaq import spin

# Parameterized circuit with theta as the parameter
kernel, theta = cudaq.make_kernel(list)
qreg = kernel.qalloc(2)
kernel.x(qreg[0])
kernel.ry(theta[0], qreg[1])

# Observable  
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0)  

# Initialize the gradient-free optimizer COBYLA
optimizer = cudaq.optimizers.COBYLA()

# Specify the number of iterations (optional)
optimizer.max_iterations = 10

def cost_function(x):
    # cudaq.observe() produces the expected value of a specified observable wrt a given parameterized ansatz at given params.
    # This value is the cost function wrt which we are optimizing.
    observeResult = cudaq.observe(kernel, hamiltonian, x)
    print (observeResult.expectation(), x)
    return observeResult.expectation()

# Carry out the optimization
opt_value, opt_theta = optimizer.optimize(dimensions=1, function=cost_function)

-0.9999999999999997 [0.0]
0.0 [1.5707963267948966]
-1.9999999999999996 [-1.5707963267948966]
-0.9999999999999997 [-3.141592653589793]
-0.9999999999999997 [0.0]
-1.7071067811865468 [-0.7853981633974483]
-1.9238795325112867 [-1.9634954084936207]
-1.9807852804032298 [-1.3744467859455345]
-1.995184726672196 [-1.6689710972195777]
-1.998795456205172 [-1.521708941582556]


    VQE wrapper : combines the expectation value computation with the optimzation part.

In [23]:
 # Import the necessary modules
 import cudaq
 from cudaq import spin
 
 # Parameterized circuit with theta as the parameter
 kernel, theta = cudaq.make_kernel(list)
 qreg = kernel.qalloc(1)
 kernel.x(qreg[0])
 kernel.ry(theta[0], qreg[0])
 
 # Hamiltonian operator 
 hamiltonian = hamiltonian = spin.z(0) 
 
 # Initialize the gradient-free optimizer COBYLA
 optimizer = cudaq.optimizers.COBYLA()
 
 # Specify the number of iterations (optional)
 optimizer.max_iterations = 5
 
 # Carry out the optimization
 opt_value, opt_theta = cudaq.vqe(kernel=kernel, 
                         spin_operator=hamiltonian,
                         optimizer=optimizer,
                         parameter_count=1)
 
 print(f"\nminimized <H> = {round(opt_value,16)}")
 print(f"optimal theta = {round(opt_theta[0],16)}")


minimized <H> = -1.0
optimal theta = 0.0


### 7. Single-gpu acceleration
    We demonstrate the power of GPU acceleration via GHZ state preparation.
    (About 200x faster than its CPU counterpart!)

In [24]:
import cudaq

cudaq.set_target("nvidia") # activates the single-gpu backend

def ghz_state(N):
    kernel = cudaq.make_kernel()
    q = kernel.qalloc(N)
    kernel.h(q[0])
    for i in range(N - 1):
      kernel.cx(q[i], q[i + 1])
 
    kernel.mz(q)
    return kernel

n = 30
print("Preparing GHZ state for", n, "qubits.")
kernel = ghz_state(n)
counts = cudaq.sample(kernel)
counts.dump()

RuntimeError: [custatevec] %CUDA driver version is insufficient for CUDA runtime version in CuStateVecCircuitSimulator (line 334)

    Important Links 

    * Installation
        https://nvidia.github.io/cuda-quantum/latest/install.html 
    * Documentation
        https://nvidia.github.io/cuda-quantum/latest/index.html
    * CUDA Quantum Repo
        https://github.com/NVIDIA/cuda-quantum
    * Scaling Applications
        https://nvidia.github.io/cuda-quantum/latest/examples/python/tutorials/multi_gpu_workflows.html

